In [1]:
# Import libraries
import os
import pandas as pd

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [2]:
# authenticate
auth.authenticate_user()

In [3]:
# Set up environment variables
project_id = 'mimic-433109'

os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

dataset = 'mimic_demo'

In [4]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/My Drive/MLA4H/project

/content/drive/My Drive/MLA4H/project


# 1. Check itemid

In [6]:
%%time

query = """
SELECT *
FROM `physionet-data.mimiciv_hosp.d_labitems`
WHERE LOWER(label) LIKE '%calcium%'
   OR LOWER(label) LIKE '%parathyroid%'
   OR LOWER(label) LIKE '%pth%'
   OR LOWER(label) LIKE '%vitamin d%'
"""

run_query(query).tail(20)

<ipython-input-3-c60b5c21247e>:8: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


CPU times: user 175 ms, sys: 30.5 ms, total: 206 ms
Wall time: 3.48 s


,itemid,label,fluid,category
0,50808,Free Calcium,Blood,Blood Gas
1,52029,% Ionized Calcium,Blood,Blood Gas
2,52034,Total Calcium,Blood,Blood Gas
3,52035,Total Calcium,Blood,Blood Gas
4,50853,25-OH Vitamin D,Blood,Chemistry
5,50893,"Calcium, Total",Blood,Chemistry
6,50965,Parathyroid Hormone,Blood,Chemistry
7,51624,Free Calcium,Blood,Chemistry
8,51066,24 hr Calcium,Urine,Chemistry
9,51077,"Calcium, Urine",Urine,Chemistry


# 2. Check icd_code & icd_version

In [7]:
%%time

query = """
SELECT *
FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`
WHERE (long_title LIKE '%Osteoporosis%')
OR (long_title LIKE '%Hyperparathyroidism%')
"""

run_query(query).tail(10)

<ipython-input-3-c60b5c21247e>:8: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


CPU times: user 17 ms, sys: 3.03 ms, total: 20 ms
Wall time: 535 ms


,icd_code,icd_version,long_title
0,25200,9,"Hyperparathyroidism, unspecified"
1,73300,9,"Osteoporosis, unspecified"
2,E21,10,Hyperparathyroidism and other disorders of par...
3,E213,10,"Hyperparathyroidism, unspecified"
4,M80,10,Osteoporosis with current pathological fracture
5,M81,10,Osteoporosis without current pathological frac...


# 3. Data Collection

In [10]:
%%time

query = f"""
WITH OpPatients AS (
    SELECT DISTINCT hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
    WHERE (icd_code = '73300' AND icd_version = 9)
    OR (icd_code = 'M80' AND icd_version = 10)
    OR (icd_code = 'M81' AND icd_version = 10)
),
HpPatients AS (
    SELECT DISTINCT hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
    WHERE (icd_code = '25200' AND icd_version = 9)
    OR (icd_code = 'E21' AND icd_version = 10)
    OR (icd_code = 'E213' AND icd_version = 10)
),
AllPatients AS (
    SELECT DISTINCT hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
),
LabEventsItem AS (
    SELECT subject_id, hadm_id, itemid, valuenum,
    FROM `physionet-data.mimiciv_hosp.labevents`
    WHERE (itemid IN (50808, 52029,52034, 52035, 50853, 51624,
    50893, 51077, 50965))
    AND (hadm_id IN (SELECT hadm_id FROM AllPatients))
),
LabEventsByHadmItem AS(
    SELECT
        subject_id,
        hadm_id,

        -- Columns for itemid 50808
        MAX(CASE WHEN itemid = 50808 THEN valuenum ELSE NULL END) AS max_FreeCalcium1,
        MIN(CASE WHEN itemid = 50808 THEN valuenum ELSE NULL END) AS min_FreeCalcium1,
        AVG(CASE WHEN itemid = 50808 THEN valuenum ELSE NULL END) AS avg_FreeCalcium1,

        -- Columns for itemid 52029
        MAX(CASE WHEN itemid = 52029 THEN valuenum ELSE NULL END) AS max_IonizedCalcium,
        MIN(CASE WHEN itemid = 52029 THEN valuenum ELSE NULL END) AS min_IonizedCalcium,
        AVG(CASE WHEN itemid = 52029 THEN valuenum ELSE NULL END) AS avg_IonizedCalcium,

        -- Columns for itemid 52034
        MAX(CASE WHEN itemid = 52034 THEN valuenum ELSE NULL END) AS max_TotalCalcium1,
        MIN(CASE WHEN itemid = 52034 THEN valuenum ELSE NULL END) AS min_TotalCalcium1,
        AVG(CASE WHEN itemid = 52034 THEN valuenum ELSE NULL END) AS avg_TotalCalcium1,

        -- Columns for itemid 52035
        MAX(CASE WHEN itemid = 52035 THEN valuenum ELSE NULL END) AS max_TotalCalcium2,
        MIN(CASE WHEN itemid = 52035 THEN valuenum ELSE NULL END) AS min_TotalCalcium2,
        AVG(CASE WHEN itemid = 52035 THEN valuenum ELSE NULL END) AS avg_TotalCalcium2,

        -- Columns for itemid 50853
        MAX(CASE WHEN itemid = 50853 THEN valuenum ELSE NULL END) AS max_25OHVitaminD,
        MIN(CASE WHEN itemid = 50853 THEN valuenum ELSE NULL END) AS min_25OHVitaminD,
        AVG(CASE WHEN itemid = 50853 THEN valuenum ELSE NULL END) AS avg_25OHVitaminD,

        -- Columns for itemid 51624
        MAX(CASE WHEN itemid = 51624 THEN valuenum ELSE NULL END) AS max_FreeCalcium2,
        MIN(CASE WHEN itemid = 51624 THEN valuenum ELSE NULL END) AS min_FreeCalcium2,
        AVG(CASE WHEN itemid = 51624 THEN valuenum ELSE NULL END) AS avg_FreeCalcium2,

        -- Columns for itemid 50893
        MAX(CASE WHEN itemid = 50893 THEN valuenum ELSE NULL END) AS max_CalciumTotal,
        MIN(CASE WHEN itemid = 50893 THEN valuenum ELSE NULL END) AS min_CalciumTotal,
        AVG(CASE WHEN itemid = 50893 THEN valuenum ELSE NULL END) AS avg_CalciumTotal,

        -- Columns for itemid 51077
        MAX(CASE WHEN itemid = 51077 THEN valuenum ELSE NULL END) AS max_CalciumUrine,
        MIN(CASE WHEN itemid = 51077 THEN valuenum ELSE NULL END) AS min_CalciumUrine,
        AVG(CASE WHEN itemid = 51077 THEN valuenum ELSE NULL END) AS avg_CalciumUrine,

        -- Columns for itemid 50965
        MAX(CASE WHEN itemid = 50965 THEN valuenum ELSE NULL END) AS max_ParathyroidHormone,
        MIN(CASE WHEN itemid = 50965 THEN valuenum ELSE NULL END) AS min_ParathyroidHormone,
        AVG(CASE WHEN itemid = 50965 THEN valuenum ELSE NULL END) AS avg_ParathyroidHormone,

    FROM LabEventsItem
    GROUP BY subject_id,hadm_id
    HAVING NOT (
        max_FreeCalcium1 IS NULL AND min_FreeCalcium1 IS NULL AND avg_FreeCalcium1 IS NULL AND
        max_IonizedCalcium IS NULL AND min_IonizedCalcium IS NULL AND avg_IonizedCalcium IS NULL AND
        max_TotalCalcium1 IS NULL AND min_TotalCalcium1 IS NULL AND avg_TotalCalcium1 IS NULL AND
        max_TotalCalcium2 IS NULL AND min_TotalCalcium2 IS NULL AND avg_TotalCalcium2 IS NULL AND
        max_25OHVitaminD IS NULL AND min_25OHVitaminD IS NULL AND avg_25OHVitaminD IS NULL AND
        max_FreeCalcium2 IS NULL AND min_FreeCalcium2 IS NULL AND avg_FreeCalcium2 IS NULL AND
        max_CalciumTotal IS NULL AND min_CalciumTotal IS NULL AND avg_CalciumTotal IS NULL AND
        max_CalciumUrine IS NULL AND min_CalciumUrine IS NULL AND avg_CalciumUrine IS NULL AND
        max_ParathyroidHormone IS NULL AND min_ParathyroidHormone IS NULL AND avg_ParathyroidHormone IS NULL
    )
),
LabEventsPatient AS(
    SELECT
        t1.*, t2.gender, t2.anchor_age
    FROM LabEventsByHadmItem AS t1
    JOIN `physionet-data.mimiciv_hosp.patients` AS t2
    ON t1.subject_id = t2.subject_id
),
LabeledLabEvents1 AS (
    SELECT
        t1.*,
        CASE WHEN t2.hadm_id IS NOT NULL THEN 1 ELSE 0 END AS Osteoporosis
    FROM LabEventsPatient AS t1
    LEFT JOIN OpPatients AS t2
    ON t1.hadm_id = t2.hadm_id
),
LabeledLabEvents2 AS (
    SELECT
        t1.max_FreeCalcium1, t1.min_FreeCalcium1, t1.avg_FreeCalcium1,
        t1.max_IonizedCalcium, t1.min_IonizedCalcium, t1.avg_IonizedCalcium,
        t1.max_TotalCalcium1, t1.min_TotalCalcium1, t1.avg_TotalCalcium1,
        t1.max_TotalCalcium2, t1.min_TotalCalcium2, t1.avg_TotalCalcium2,
        t1.max_25OHVitaminD, t1.min_25OHVitaminD, t1.avg_25OHVitaminD,
        t1.max_FreeCalcium2, t1.min_FreeCalcium2, t1.avg_FreeCalcium2,
        t1.max_CalciumTotal, t1.min_CalciumTotal, t1.avg_CalciumTotal,
        t1.max_CalciumUrine, t1.min_CalciumUrine, t1.avg_CalciumUrine,
        t1.max_ParathyroidHormone, t1.min_ParathyroidHormone, t1.avg_ParathyroidHormone,
        t1.anchor_age,
        t1.gender AS gender,
        t1.Osteoporosis,
        CASE WHEN t2.hadm_id IS NOT NULL THEN 1 ELSE 0 END AS Hyperparathyroidism
    FROM LabeledLabEvents1 AS t1
    LEFT JOIN HpPatients AS t2
    ON t1.hadm_id = t2.hadm_id
)

SELECT * FROM LabeledLabEvents2
"""

df = run_query(query)
df.tail(10)

<ipython-input-3-c60b5c21247e>:8: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  return pd.io.gbq.read_gbq(


CPU times: user 26.2 s, sys: 387 ms, total: 26.6 s
Wall time: 1min 26s


,max_FreeCalcium1,min_FreeCalcium1,avg_FreeCalcium1,max_IonizedCalcium,min_IonizedCalcium,avg_IonizedCalcium,max_TotalCalcium1,min_TotalCalcium1,avg_TotalCalcium1,max_TotalCalcium2,...,max_CalciumUrine,min_CalciumUrine,avg_CalciumUrine,max_ParathyroidHormone,min_ParathyroidHormone,avg_ParathyroidHormone,anchor_age,gender,Osteoporosis,Hyperparathyroidism
290496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,72,F,0,0
290497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,62,M,0,0
290498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,57,M,0,0
290499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,67,F,1,0
290500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,47,F,0,0
290501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,34,M,0,0
290502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,85,F,1,0
290503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,50,F,0,0
290504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,57,M,0,0
290505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,64,F,0,0


In [11]:
# Save DataFrame to CSV
df.to_csv('../data/raw_Op_Hp_LabEvents.csv', index=False)